# Italian Airbnb's: An Analytics Showcase

## The Beginning
As someone who studied and lived in southern Switzerland for four years, just minutes from the Italian border, and who learned Italian to a C1 level, I have a fond place in my heart for Italy and Italian culture. As a well-traveled person with an insatiable curiousity, I wanted to examine Italian AirBnB rentals for insights I might glean, while learning and showing off my SQL and Python skills.

I found the primary dataset for this on kaggle __[here](https://www.kaggle.com/datasets/salvatoremarcello/italian-airbnb-dataset)__. Other AirBnB datasets of interest reside __[here](https://insideairbnb.com/get-the-data/)__, and will bear some investigation and incorporation later.

## Prepping our data for load

While the structures of this git repository and associated databases are intended to mimic a potential production setup, certain elements (such as the absence of scheduled ETL) are missing. The focus of this project and associated resources is to showcase SQL competency, and as a consequence, some infrastructural knowledge around OLAP database structure. In our case, the overall structure is as follows:
1. A jupyter notebook (this one) cleans our csv
2. DDL scripts are run to prep a data mart for loading
3. A staging table is loaded from cleaned csv via a bash script
4. The staging table is then written to the data mart and erased

<div class="alert alert-block alert-info">
Postgres may not be the ideal OLAP RDBMS, but for this use case it works sufficiently well. It was chosen for many reasons, among them: 1. PostgreSQL is common and PL/pgSQL is a familiar dialect that lets us emulate an ROLAP environment like Redshift 2. Quick deploy with bitnami helm chart for availability even locally 3. It integrates with Tableau
</div>

***

In [1]:
# Load in packages
import pandas as pd
import numpy as np
# Constants import for brevity
from constants import *
# Setting max display
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
# Let's load in our dataset.
exp = pd.read_csv("~/projects/airbnb_sql/op-db/airbnb.csv")

### Some investigation

I want to see what I'm working with here, so I look at the cities, how many unique instances of "how long our hosts have been around" there are, and what dates our scrapings occurred. This is to get a feel for the data and its cleanliness. Thankfully, the data is already relatively clean. Finally, I look at 10 rows. (Condensed from 100 for clarity + brevity)

In [4]:
# Printing our unique values
print(pd.unique(exp['City']))
print(len(pd.unique(exp['Host since'])))
print(pd.unique(exp['Date of scraping']))
# Taking the head
exp.head(10)

['Firenze' 'Milano' 'Napoli' 'Roma' 'Venezia']
5074
['2024-12-15' '2024-03-15' '2024-06-15' '2024-09-15']


,Listings id,Last year reviews,Host since,Host is superhost,Host number of listings,Neighbourhood,Beds number,Bedrooms number,Property type,Maximum allowed guests,Price,Total reviews,Rating score,Accuracy score,Cleanliness score,Checkin score,Communication score,Location score,Value for money score,Reviews per month,City,Season,Bathrooms number,Bathrooms type,Coordinates,Date of scraping
0,31840,6,2011-02-07,Host,44.0,Centro Storico,1.0,1.0,Private room,2,89.0,128,4.65,4.73,4.86,4.85,4.91,4.91,4.70,0.78,Firenze,Early Winter,1,private,"43.77709, 11.25216",2024-12-15
1,222527,0,2011-07-11,Superhost,3.0,Rifredi,3.0,2.0,Entire home,4,300.0,284,4.85,4.88,4.88,4.99,4.96,4.60,4.86,1.92,Firenze,Early Winter,1,private,"43.82005, 11.22004",2024-12-15
2,32120,6,2010-03-26,Host,1.0,Gavinana Galluzzo,1.0,1.0,Entire home,2,95.0,26,4.88,4.88,4.88,4.84,4.96,4.63,4.63,0.16,Firenze,Early Winter,1,private,"43.76157, 11.27741",2024-12-15
3,224562,9,2011-09-16,Host,2.0,Centro Storico,1.0,1.0,Entire home,4,60.0,101,4.66,4.75,4.71,4.76,4.83,4.93,4.64,0.71,Firenze,Early Winter,1,private,"43.772, 11.26142",2024-12-15
4,32180,11,2014-04-05,Superhost,1.0,Centro Storico,4.0,2.0,Entire home,4,105.0,34,4.76,4.81,4.84,4.81,4.71,4.81,4.84,0.21,Firenze,Early Winter,2,private,"43.76832, 11.24348",2024-12-15
5,224669,17,2011-09-16,Superhost,3.0,Campo di Marte,2.0,1.0,Entire home,4,86.0,57,5.00,5.00,5.00,4.98,4.98,4.96,4.98,0.86,Firenze,Early Winter,1,private,"43.76796, 11.2714",2024-12-15
6,32395,80,2010-06-07,Host,6.0,Centro Storico,1.0,1.0,Hotel room,2,63.0,682,4.84,4.83,4.85,4.90,4.92,4.75,4.83,4.19,Firenze,Early Winter,1,private,"43.76268, 11.24149",2024-12-15
7,227059,67,2011-09-20,Superhost,2.0,Centro Storico,1.0,1.0,Private room,2,35.0,461,4.77,4.80,4.76,4.89,4.86,4.98,4.80,3.45,Firenze,Early Winter,2,shared,"43.77022, 11.2561",2024-12-15
8,39115,13,2010-07-15,Host,12.0,Centro Storico,2.0,1.0,Private room,2,83.0,76,4.55,4.70,4.63,4.83,4.76,4.91,4.56,0.47,Firenze,Early Winter,0,private,"43.77525, 11.25258",2024-12-15
9,230853,53,2011-09-25,Superhost,1.0,Campo di Marte,2.0,1.0,Private room,2,61.0,549,4.85,4.88,4.89,4.93,4.90,4.58,4.83,3.91,Firenze,Early Winter,1,private,"43.77388, 11.2844",2024-12-15


### Observations

Our data looks quite clean from initial observations, and we have many diverse hosts with different start dates. There is a lot to work with here, so let's dive into examining what could make for good keys in our Postgres data mart.

The first step is looking at Listings id as a potential key - how many unique ones are there? Does that correspond to the number of observations in our data set? Printing the below values we see it would make for a poor primary key, so we will likely just go with surrogates in our data mart.

Let's dive into this a bit more, since I assumed that Listing IDs would have more unique values. How many repeated observations do we have in our dataset, if we look at Listings IDs? In this case I drilled down to one listing ID and found all attributes were the same with the exception of `scraping season` and `reviews per month`. So, it seems many listings have multiple scraping dates, with potentially different scores across Accuracy, Cleanliness, Checkin, Communication, Location, etc. We can further verify our listing id as the same location because of the coordinates included in the dataset.

In [5]:
# Print number of observations in our dataset
print(len(exp))
# Print how many unique Listing IDs there are in our dataset.
print(len(pd.unique(exp['Listings id'])))

282047
92355


In [6]:
# Let's see what repeated observations we have in the dataset
repeated_observations = exp[exp['Listings id'].duplicated(keep=False)]
# Let's see what 222527 as a Listing ID looks like, for an idea of the duplication that is happening
repeated_observations[repeated_observations['Listings id'] == 222527]

,Listings id,Last year reviews,Host since,Host is superhost,Host number of listings,Neighbourhood,Beds number,Bedrooms number,Property type,Maximum allowed guests,Price,Total reviews,Rating score,Accuracy score,Cleanliness score,Checkin score,Communication score,Location score,Value for money score,Reviews per month,City,Season,Bathrooms number,Bathrooms type,Coordinates,Date of scraping
1,222527,0,2011-07-11,Superhost,3.0,Rifredi,3.0,2.0,Entire home,4,300.0,284,4.85,4.88,4.88,4.99,4.96,4.6,4.86,1.92,Firenze,Early Winter,1,private,"43.82005, 11.22004",2024-12-15
14658,222527,0,2011-07-11,Superhost,3.0,Rifredi,3.0,1.0,Entire home,4,300.0,284,4.85,4.88,4.88,4.99,4.96,4.6,4.86,1.88,Firenze,Early Spring,1,private,"43.82005, 11.22004",2024-03-15


### Coordinate Handling and Column renaming

A quick look at Florence to help us investigate the dataset further reveals a pretty clean dataset that seems ready for load to our DB. 

We convert the coordinates into WKT format for easy loading to Postgres; WKT represents spatial data as a text string, such as `'POINT(x y)'` for a single coordinate. We then rename and shuffle columns to align with our staging table for easy loading via COPY command in our bash script. This process parallels the AWS recommended load method for OLAP Redshift in AWS from S3 object storage.

In [7]:
exp[exp["City"] == "Firenze"]

,Listings id,Last year reviews,Host since,Host is superhost,Host number of listings,Neighbourhood,Beds number,Bedrooms number,Property type,Maximum allowed guests,Price,Total reviews,Rating score,Accuracy score,Cleanliness score,Checkin score,Communication score,Location score,Value for money score,Reviews per month,City,Season,Bathrooms number,Bathrooms type,Coordinates,Date of scraping
0,31840,6,2011-02-07,Host,44.0,Centro Storico,1.0,1.0,Private room,2,89.0,128,4.65,4.73,4.86,4.85,4.91,4.91,4.70,0.78,Firenze,Early Winter,1,private,"43.77709, 11.25216",2024-12-15
1,222527,0,2011-07-11,Superhost,3.0,Rifredi,3.0,2.0,Entire home,4,300.0,284,4.85,4.88,4.88,4.99,4.96,4.60,4.86,1.92,Firenze,Early Winter,1,private,"43.82005, 11.22004",2024-12-15
2,32120,6,2010-03-26,Host,1.0,Gavinana Galluzzo,1.0,1.0,Entire home,2,95.0,26,4.88,4.88,4.88,4.84,4.96,4.63,4.63,0.16,Firenze,Early Winter,1,private,"43.76157, 11.27741",2024-12-15
3,224562,9,2011-09-16,Host,2.0,Centro Storico,1.0,1.0,Entire home,4,60.0,101,4.66,4.75,4.71,4.76,4.83,4.93,4.64,0.71,Firenze,Early Winter,1,private,"43.772, 11.26142",2024-12-15
4,32180,11,2014-04-05,Superhost,1.0,Centro Storico,4.0,2.0,Entire home,4,105.0,34,4.76,4.81,4.84,4.81,4.71,4.81,4.84,0.21,Firenze,Early Winter,2,private,"43.76832, 11.24348",2024-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41238,1236248338760723102,1,2024-09-01,Host,1.0,Campo di Marte,2.0,2.0,Entire home,4,105.0,1,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00,Firenze,Early Autumn,1,private,"43.7689407, 11.2797417",2024-09-15
41239,1237418137104849814,2,2019-04-16,Host,5.0,Centro Storico,3.0,2.0,Entire home,4,125.0,2,4.50,4.00,5.00,3.50,3.00,3.00,3.50,2.00,Firenze,Early Autumn,1,private,"43.7825537562829, 11.23673893236205",2024-09-15
41240,1238313522110505122,1,2024-09-04,Host,1.0,Campo di Marte,1.0,1.0,Entire home,2,98.0,1,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00,Firenze,Early Autumn,1,private,"43.788893, 11.2743626",2024-09-15
41241,1239066145299909903,1,2012-05-20,Host,1.0,Centro Storico,1.0,1.0,Private room,1,94.0,1,5.00,5.00,5.00,5.00,5.00,4.00,5.00,1.00,Firenze,Early Autumn,1,shared,"43.78409984948205, 11.230298367350713",2024-09-15


In [8]:
# Import shapely to convert our coordinate to WKT
from shapely.geometry import Point

def to_wkt(coord):
    lat, lon = map(float, coord.split(','))
    return Point(lon, lat).wkt  # Note: WKT format is (lon lat)
# WKT conversion
exp['Coordinates'] = exp['Coordinates'].apply(to_wkt)

# Rename the columns and reorder using constants
exp = exp.rename(columns=column_mapping)
exp = exp[staging_columns]
# Convert Host is superhost to boolean for performance.
exp['host_is_superhost'] = exp['host_is_superhost'].map({'Superhost': True, 'Host': False})

### Saving
Now that we have it clean, let's save our CSV.

**An aside about cloud vs on-premise**: If I were to set up a consistent ETL for Airbnb datasets such as this one on a batch monthly basis in AWS, I would automate this process with Airflow in EKS and a docker container for extraction + cleaning, transforming, and loading with a combination of KubernetesPodOperator, BashOperator, and PostgresOperator as necessary. I would deploy the Airflow in EKS with a replicable Helm chart and utilize S3 as the landing area for data ingestion and pre-load to staging. I would also be using parquet format to optimize costs (in the scenario of terabyte or petabyte-scale ingestion).

Additionally, I would investigate the use of a parquet based Iceberg Lakehouse in S3 with a Glue Data Catalog, since Tableau has easy integration with S3.

<u>I'm doing all this here because I am focusing on showcasing my SQL and Analytics skills. My other projects involve cloud infrastructure deployed with Hashicorp's Terraform for replicability.</u>


In [ ]:
exp.to_csv("/home/eandrews/projects/de-proj-1/op-db/airbnb_clean.csv", index=False)